In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random

import nltk
from nltk.stem import WordNetLemmatizer

import re

from spacy.tokens import DocBin

from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\train.csv")
features = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\features.csv")
patient_notes = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\patient_notes.csv")

### CASE - 0

In [3]:
case_0_features = features[features["case_num"] == 0]
case_0_features

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
5,5,0,No-hair-changes-OR-no-nail-changes-OR-no-tempe...
6,6,0,Adderall-use
7,7,0,Shortness-of-breath
8,8,0,Caffeine-use
9,9,0,heart-pounding-OR-heart-racing


In [4]:
case_0_pn = patient_notes[patient_notes["case_num"] ==  0]
case_0_pn

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...
...,...,...,...
2263,2447,0,17 yo M comes to clinic c/o of 1st time episod...
2264,2448,0,17 yo m c/o heart pounding for the past 2-3 mo...
2265,2449,0,Pt is a 17yM with c/o Heart pounding\r\n- Onse...
2266,2450,0,17 year old male presents with heart pounding ...


In [5]:
case_0_train = train_data[train_data["case_num"] == 0]
case_0_train = case_0_train[case_0_train.annotation != '[]']
case_0_train.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


### Over Sampling for CASE - 0

In [6]:
pn_dict = {}
for idx, row in case_0_pn.iterrows():
    pn_dict[row['pn_num']] = row['pn_history']

In [7]:
new_annotation = []
for case_id in case_0_features['case_num'].unique():
    
    all_pn_id = set(case_0_pn[case_0_pn['case_num']==case_id]['pn_num'].tolist())
    
    for feature_id in case_0_features[case_0_features['case_num']==case_id]['feature_num'].unique():
        # get all the pn_num that have already been annotated
        annotated_pn = set(case_0_train[case_0_train['feature_num']==feature_id]['pn_num'].tolist())
        # get all the pn_num that have NOT been annotated
        pn_to_annotate = all_pn_id-annotated_pn
        
        # get all current annotations
        # we will use them to find more annotations
        annotations = case_0_train[case_0_train['feature_num']==feature_id]['annotation'].tolist()
        annotation_texts = set()
        for a in annotations:
            anns = eval(a)
            for at in anns:
                annotation_texts.add(at)
                
        # annotate       
        for pn_id in pn_to_annotate:
            new_annotation_pn, new_location_pn = [], []
            pn_text = pn_dict[pn_id]
            for at in annotation_texts:
                start = pn_text.find(at)
                if start>=0:
                    new_annotation_pn.append(at)
                    new_location_pn.append(f'{start} {start+len(at)}')
            if len(new_annotation_pn)>0:
                new_annotation.append((
                    f'{pn_id:04d}_{feature_id:03d}',
                    case_id,
                    pn_id,
                    feature_id,
                    new_annotation_pn,
                    new_location_pn
                ))
     #   break
    break
    # break to get sample results quickly

In [8]:
len(new_annotation)

18810

In [9]:
df=pd.DataFrame(new_annotation)
df.head()

,0,1,2,3,4,5
0,0000_000,0,0,0,[father had MI],[553 566]
1,0005_000,0,5,0,[father - MI],[825 836]
2,0007_000,0,7,0,[Father MI],[490 499]
3,0009_000,0,9,0,[father had MI],[709 722]
4,0011_000,0,11,0,[Father had MI],[684 697]


In [10]:
df.columns=['id','case_num','pn_num','feature_num','annotation','location']
df.head()

,id,case_num,pn_num,feature_num,annotation,location
0,0000_000,0,0,0,[father had MI],[553 566]
1,0005_000,0,5,0,[father - MI],[825 836]
2,0007_000,0,7,0,[Father MI],[490 499]
3,0009_000,0,9,0,[father had MI],[709 722]
4,0011_000,0,11,0,[Father had MI],[684 697]


In [11]:
df["location"] = df["location"].apply(str)
df["annotation"] = df["annotation"].apply(str)

df["annotation"] = df.annotation.str.replace('[','')
df["annotation"] = df.annotation.str.replace(']','')
df["location"] = df.location.str.replace('[','')
df["location"] = df.location.str.replace(']','')

df.head()

,id,case_num,pn_num,feature_num,annotation,location
0,0000_000,0,0,0,'father had MI','553 566'
1,0005_000,0,5,0,'father - MI','825 836'
2,0007_000,0,7,0,'Father MI','490 499'
3,0009_000,0,9,0,'father had MI','709 722'
4,0011_000,0,11,0,'Father had MI','684 697'


In [12]:
df = df[(df["feature_num"] == 4)|(df["feature_num"] == 5)|(df["feature_num"] == 8)|(df["feature_num"] == 9)][['pn_num','annotation',"location","feature_num"]]

In [13]:
df = (df.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(df.columns, axis=1))
df

,pn_num,annotation,location,feature_num
0,1,'light headed','179 191',4
1,1,'light headedness','179 195',4
2,2,'lightheadedness','430 445',4
3,2,'lightheaded','430 441',4
4,5,'lightheadedness','204 219',4
...,...,...,...,...
7158,2451,'palpitations','118 130',9
7159,2451,'palpitation','118 129',9
7160,2451,'heart palpitation','112 129',9
7161,2451,'heart pounding','4 18',9


In [14]:
df["start_location"] = df["location"].apply(lambda x: x.split()[0][1:])
df["end_location"] = df["location"].apply(lambda x: x.split()[-1][:-1])
df

,pn_num,annotation,location,feature_num,start_location,end_location
0,1,'light headed','179 191',4,179,191
1,1,'light headedness','179 195',4,179,195
2,2,'lightheadedness','430 445',4,430,445
3,2,'lightheaded','430 441',4,430,441
4,5,'lightheadedness','204 219',4,204,219
...,...,...,...,...,...,...
7158,2451,'palpitations','118 130',9,118,130
7159,2451,'palpitation','118 129',9,118,129
7160,2451,'heart palpitation','112 129',9,112,129
7161,2451,'heart pounding','4 18',9,4,18


In [15]:
df_4 = df[df["feature_num"] == 4].sample(n = 60, random_state = 100)
df_4.shape

(60, 6)

In [16]:
df_5 = df[df["feature_num"] == 5].sample(n = 70, random_state = 100)
df_5.shape

(70, 6)

In [17]:
df_8 = df[df["feature_num"] == 8].sample(n = 80, random_state = 100)
df_8.shape

(80, 6)

In [18]:
df_9 = df[df["feature_num"] == 9].sample(n = 30, random_state = 100)
df_9.shape

(30, 6)

### Case - 0 Train data

In [19]:
case_0_train = case_0_train[['pn_num','annotation',"location","feature_num"]]

In [20]:
case_0_train["annotation"] = case_0_train.annotation.str.replace('[','')
case_0_train["annotation"] = case_0_train.annotation.str.replace(']','')
case_0_train["location"] = case_0_train.location.str.replace('[','')
case_0_train["location"] = case_0_train.location.str.replace(']','')

case_0_train

,pn_num,annotation,location,feature_num
0,16,'dad with recent heart attcak','696 724',0
1,16,"'mom with ""thyroid disease'",'668 693',1
2,16,'chest pressure','203 217',2
3,16,"'intermittent episodes', 'episode'","'70 91', '176 183'",3
4,16,'felt as if he were going to pass out','222 258',4
...,...,...,...,...
1294,2436,'shortness of breath','371 390',7
1296,2436,"'chest palpitations', 'chest palpitations'","'71 89', '109 127'",9
1297,2436,'2-3 month','50 59',10
1298,2436,'17 year old','26 37',11


In [21]:
case_0_train.loc[case_0_train["annotation"] == "'dad, heart attack'","annotation"] = "'dad  heart attack'"
case_0_train

,pn_num,annotation,location,feature_num
0,16,'dad with recent heart attcak','696 724',0
1,16,"'mom with ""thyroid disease'",'668 693',1
2,16,'chest pressure','203 217',2
3,16,"'intermittent episodes', 'episode'","'70 91', '176 183'",3
4,16,'felt as if he were going to pass out','222 258',4
...,...,...,...,...
1294,2436,'shortness of breath','371 390',7
1296,2436,"'chest palpitations', 'chest palpitations'","'71 89', '109 127'",9
1297,2436,'2-3 month','50 59',10
1298,2436,'17 year old','26 37',11


In [22]:
case_0_train = (case_0_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(case_0_train.columns, axis=1))
case_0_train

,pn_num,annotation,location,feature_num
0,16,'dad with recent heart attcak','696 724',0
1,16,"'mom with ""thyroid disease'",'668 693',1
2,16,'chest pressure','203 217',2
3,16,'intermittent episodes','70 91',3
4,16,'episode','176 183',3
...,...,...,...,...
1198,2436,'chest palpitations','71 89',9
1199,2436,'chest palpitations','109 127',9
1200,2436,'2-3 month','50 59',10
1201,2436,'17 year old','26 37',11


In [23]:
case_0_train["start_location"] = case_0_train["location"].apply(lambda x: x.split()[0][1:])
case_0_train["end_location"] = case_0_train["location"].apply(lambda x: x.split()[-1][:-1])
case_0_train

,pn_num,annotation,location,feature_num,start_location,end_location
0,16,'dad with recent heart attcak','696 724',0,696,724
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693
2,16,'chest pressure','203 217',2,203,217
3,16,'intermittent episodes','70 91',3,70,91
4,16,'episode','176 183',3,176,183
...,...,...,...,...,...,...
1198,2436,'chest palpitations','71 89',9,71,89
1199,2436,'chest palpitations','109 127',9,109,127
1200,2436,'2-3 month','50 59',10,50,59
1201,2436,'17 year old','26 37',11,26,37


In [24]:
case_0 = pd.concat([case_0_train, df_4, df_5, df_8,df_9])
case_0.shape

(1443, 6)

In [25]:
case_0 = case_0.merge(patient_notes, on='pn_num', how='left')
case_0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...
...,...,...,...,...,...,...,...,...
1438,520,'palpitation','30 41',9,30,41,0,Consent givenbymom17 yo M c/o palpitations sin...
1439,2351,'palpitations','65 77',9,65,77,0,17 year old male who presents with several mon...
1440,563,'palpitations','24 36',9,24,36,0,17 yo M presenting with palpitations. This has...
1441,1840,'heart is going to jump out of his chest','104 143',9,104,143,0,Mr. Cleveland is a 17 y/o M presenting with 2-...


In [26]:
case_0["New_annotation"] = case_0.apply(lambda x: x.pn_history[int(x.start_location):int(x.end_location)],axis=1)
case_0.head()

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,"mom with ""thyroid disease"
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episodes
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode


In [27]:
case_0["New_annotation"] = case_0["New_annotation"].apply(lambda x: re.sub(r'''[/"+,()\r\n]''',' ',x))
case_0["New_annotation"] = case_0["New_annotation"].apply(lambda x: re.sub(r'''[']''','',x))
case_0["New_annotation"] = case_0["New_annotation"].apply(lambda x: x.strip())
case_0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episodes
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode
...,...,...,...,...,...,...,...,...,...
1438,520,'palpitation','30 41',9,30,41,0,Consent givenbymom17 yo M c/o palpitations sin...,palpitation
1439,2351,'palpitations','65 77',9,65,77,0,17 year old male who presents with several mon...,palpitations
1440,563,'palpitations','24 36',9,24,36,0,17 yo M presenting with palpitations. This has...,palpitations
1441,1840,'heart is going to jump out of his chest','104 143',9,104,143,0,Mr. Cleveland is a 17 y/o M presenting with 2-...,heart is going to jump out of his chest


In [28]:
case_0["New_pn_history"] = case_0["pn_history"].apply(lambda x: re.sub(r'''[/",+()\r\n]''',' ',x))
case_0["New_pn_history"] = case_0["New_pn_history"].apply(lambda x: re.sub(r'''[']''','',x))
case_0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI: 17yo M presents with palpitations. Patien...
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI: 17yo M presents with palpitations. Patien...
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure,HPI: 17yo M presents with palpitations. Patien...
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episodes,HPI: 17yo M presents with palpitations. Patien...
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode,HPI: 17yo M presents with palpitations. Patien...
...,...,...,...,...,...,...,...,...,...,...
1438,520,'palpitation','30 41',9,30,41,0,Consent givenbymom17 yo M c/o palpitations sin...,palpitation,Consent givenbymom17 yo M c o palpitations sin...
1439,2351,'palpitations','65 77',9,65,77,0,17 year old male who presents with several mon...,palpitations,17 year old male who presents with several mon...
1440,563,'palpitations','24 36',9,24,36,0,17 yo M presenting with palpitations. This has...,palpitations,17 yo M presenting with palpitations. This has...
1441,1840,'heart is going to jump out of his chest','104 143',9,104,143,0,Mr. Cleveland is a 17 y/o M presenting with 2-...,heart is going to jump out of his chest,Mr. Cleveland is a 17 y o M presenting with 2-...


In [29]:
case_0["New_annotation"] = case_0.apply(lambda x : x.New_annotation.replace("SOB", "short of breath"), axis = 1)
case_0["New_pn_history"] = case_0.apply(lambda x : x.New_pn_history.replace("SOB", "short of breath"), axis = 1)
case_0["New_pn_history"] = case_0.apply(lambda x : x.New_pn_history.replace("palpitation", "heart pounding"), axis = 1)
case_0["New_annotation"] = case_0.apply(lambda x : x.New_annotation.replace("palpitation", "heart pounding"), axis = 1)
case_0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI: 17yo M presents with heart poundings. Pat...
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI: 17yo M presents with heart poundings. Pat...
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure,HPI: 17yo M presents with heart poundings. Pat...
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episodes,HPI: 17yo M presents with heart poundings. Pat...
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode,HPI: 17yo M presents with heart poundings. Pat...
...,...,...,...,...,...,...,...,...,...,...
1438,520,'palpitation','30 41',9,30,41,0,Consent givenbymom17 yo M c/o palpitations sin...,heart pounding,Consent givenbymom17 yo M c o heart poundings ...
1439,2351,'palpitations','65 77',9,65,77,0,17 year old male who presents with several mon...,heart poundings,17 year old male who presents with several mon...
1440,563,'palpitations','24 36',9,24,36,0,17 yo M presenting with palpitations. This has...,heart poundings,17 yo M presenting with heart poundings. This ...
1441,1840,'heart is going to jump out of his chest','104 143',9,104,143,0,Mr. Cleveland is a 17 y/o M presenting with 2-...,heart is going to jump out of his chest,Mr. Cleveland is a 17 y o M presenting with 2-...


In [30]:
def word_lemm(sentence):
    word_list = nltk.word_tokenize(sentence)
    
    lemmaztier = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmaztier.lemmatize(w) for w in word_list])
    return(lemmatized_output)

In [31]:
case_0["New_annotation"] = case_0["New_annotation"].apply(word_lemm)
case_0["New_pn_history"] = case_0["New_pn_history"].apply(word_lemm)
case_0.head()

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with heart pounding . Pat...
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with heart pounding . Pat...
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure,HPI : 17yo M present with heart pounding . Pat...
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episode,HPI : 17yo M present with heart pounding . Pat...
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode,HPI : 17yo M present with heart pounding . Pat...


In [32]:
case_0["new_location"] = case_0.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history),axis =1)
case_0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(666, 694), match='dad ..."
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(641, 665), match='mom ..."
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(200, 214), match='ches..."
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episode,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(71, 91), match='interm..."
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(84, 91), match='episode'>"
...,...,...,...,...,...,...,...,...,...,...,...
1438,520,'palpitation','30 41',9,30,41,0,Consent givenbymom17 yo M c/o palpitations sin...,heart pounding,Consent givenbymom17 yo M c o heart pounding s...,"<re.Match object; span=(30, 44), match='heart ..."
1439,2351,'palpitations','65 77',9,65,77,0,17 year old male who presents with several mon...,heart pounding,17 year old male who present with several mont...,"<re.Match object; span=(62, 76), match='heart ..."
1440,563,'palpitations','24 36',9,24,36,0,17 yo M presenting with palpitations. This has...,heart pounding,17 yo M presenting with heart pounding . This ...,"<re.Match object; span=(24, 38), match='heart ..."
1441,1840,'heart is going to jump out of his chest','104 143',9,104,143,0,Mr. Cleveland is a 17 y/o M presenting with 2-...,heart is going to jump out of his chest,Mr. Cleveland is a 17 y o M presenting with 2-...,"<re.Match object; span=(104, 143), match='hear..."


In [33]:
drop_index = []
for  index,row in case_0.iterrows():
    if row["new_location"] == None:
        drop_index.append(index)
        
print(drop_index),len(drop_index)

[319, 375, 476, 477, 582, 634, 635, 777, 937, 1195, 1203, 1206, 1207, 1209, 1214, 1216, 1217, 1218, 1221, 1227, 1229, 1230, 1240, 1244, 1251, 1253, 1255, 1283, 1319, 1334]


(None, 30)

In [34]:
case_0 = case_0.drop(drop_index,axis = 0)
case_0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(666, 694), match='dad ..."
1,16,"'mom with ""thyroid disease'",'668 693',1,668,693,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(641, 665), match='mom ..."
2,16,'chest pressure','203 217',2,203,217,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(200, 214), match='ches..."
3,16,'intermittent episodes','70 91',3,70,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episode,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(71, 91), match='interm..."
4,16,'episode','176 183',3,176,183,0,HPI: 17yo M presents with palpitations. Patien...,episode,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(84, 91), match='episode'>"
...,...,...,...,...,...,...,...,...,...,...,...
1438,520,'palpitation','30 41',9,30,41,0,Consent givenbymom17 yo M c/o palpitations sin...,heart pounding,Consent givenbymom17 yo M c o heart pounding s...,"<re.Match object; span=(30, 44), match='heart ..."
1439,2351,'palpitations','65 77',9,65,77,0,17 year old male who presents with several mon...,heart pounding,17 year old male who present with several mont...,"<re.Match object; span=(62, 76), match='heart ..."
1440,563,'palpitations','24 36',9,24,36,0,17 yo M presenting with palpitations. This has...,heart pounding,17 yo M presenting with heart pounding . This ...,"<re.Match object; span=(24, 38), match='heart ..."
1441,1840,'heart is going to jump out of his chest','104 143',9,104,143,0,Mr. Cleveland is a 17 y/o M presenting with 2-...,heart is going to jump out of his chest,Mr. Cleveland is a 17 y o M presenting with 2-...,"<re.Match object; span=(104, 143), match='hear..."


In [35]:
case_0["start_location"] = case_0.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).start(),axis =1)
case_0["end_location"] = case_0.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).end(),axis =1)
case_0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'dad with recent heart attcak','696 724',0,666,694,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(666, 694), match='dad ..."
1,16,"'mom with ""thyroid disease'",'668 693',1,641,665,0,HPI: 17yo M presents with palpitations. Patien...,mom with thyroid disease,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(641, 665), match='mom ..."
2,16,'chest pressure','203 217',2,200,214,0,HPI: 17yo M presents with palpitations. Patien...,chest pressure,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(200, 214), match='ches..."
3,16,'intermittent episodes','70 91',3,71,91,0,HPI: 17yo M presents with palpitations. Patien...,intermittent episode,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(71, 91), match='interm..."
4,16,'episode','176 183',3,84,91,0,HPI: 17yo M presents with palpitations. Patien...,episode,HPI : 17yo M present with heart pounding . Pat...,"<re.Match object; span=(84, 91), match='episode'>"
...,...,...,...,...,...,...,...,...,...,...,...
1438,520,'palpitation','30 41',9,30,44,0,Consent givenbymom17 yo M c/o palpitations sin...,heart pounding,Consent givenbymom17 yo M c o heart pounding s...,"<re.Match object; span=(30, 44), match='heart ..."
1439,2351,'palpitations','65 77',9,62,76,0,17 year old male who presents with several mon...,heart pounding,17 year old male who present with several mont...,"<re.Match object; span=(62, 76), match='heart ..."
1440,563,'palpitations','24 36',9,24,38,0,17 yo M presenting with palpitations. This has...,heart pounding,17 yo M presenting with heart pounding . This ...,"<re.Match object; span=(24, 38), match='heart ..."
1441,1840,'heart is going to jump out of his chest','104 143',9,104,143,0,Mr. Cleveland is a 17 y/o M presenting with 2-...,heart is going to jump out of his chest,Mr. Cleveland is a 17 y o M presenting with 2-...,"<re.Match object; span=(104, 143), match='hear..."


In [36]:
case_0 = case_0.drop(index = 161, axis = 0)
case_0 = case_0.drop(index = 87, axis = 0)
case_0 = case_0.drop(index = 164, axis = 0)
case_0 = case_0.drop(index = 248, axis = 0)
case_0 = case_0.drop(index = 4, axis = 0)
case_0 = case_0.drop(index = 76, axis = 0)
case_0 = case_0.drop(index = 127, axis = 0)
case_0 = case_0.drop(index = 141, axis = 0)
case_0 = case_0.drop(index = 197, axis = 0)
case_0 = case_0.drop(index = [231,689], axis = 0)
case_0 = case_0.drop(index = [288,389,390], axis = 0)
case_0 = case_0.drop(index = [439,438], axis = 0)
case_0 = case_0.drop(index = 534, axis = 0)
case_0 = case_0.drop(index = 547, axis = 0)
case_0 = case_0.drop(index = 575, axis = 0)
case_0 = case_0.drop(index = 714, axis = 0)
case_0 = case_0.drop(index = [728,1141], axis = 0)
case_0 = case_0.drop(index = 847, axis = 0)
case_0 = case_0.drop(index = [980,979], axis = 0)
case_0 = case_0.drop(index = 176, axis = 0)
case_0 = case_0.drop(index = 251, axis = 0)
case_0 = case_0.drop(index = 307, axis = 0)
case_0 = case_0.drop(index = 550, axis = 0)
case_0 = case_0.drop(index = 604, axis = 0)
case_0 = case_0.drop(index = 617, axis = 0)
case_0 = case_0.drop(index = 693, axis = 0)
case_0 = case_0.drop(index = 1072, axis = 0)
case_0 = case_0.drop(index = 8, axis = 0)
case_0 = case_0.drop(index = [78,77], axis = 0)
case_0 = case_0.drop(index = 187, axis = 0)
case_0 = case_0.drop(index = [236,237], axis = 0)
case_0 = case_0.drop(index = 279, axis = 0)
case_0 = case_0.drop(index = [309,310], axis = 0)
case_0 = case_0.drop(index = 338, axis = 0)
case_0 = case_0.drop(index = [348,349], axis = 0)
case_0 = case_0.drop(index = 393, axis = 0)
case_0 = case_0.drop(index = 471, axis = 0)
case_0 = case_0.drop(index = 589, axis = 0)
case_0 = case_0.drop(index = 732, axis = 0)
case_0 = case_0.drop(index = 747, axis = 0)
case_0 = case_0.drop(index = 821, axis = 0)
case_0 = case_0.drop(index = 874, axis = 0)
case_0 = case_0.drop(index = 948, axis = 0)
case_0 = case_0.drop(index = 960, axis = 0)
case_0 = case_0.drop(index = 1108, axis = 0)
case_0 = case_0.drop(index = 119, axis = 0)
case_0 = case_0.drop(index = 267, axis = 0)
case_0 = case_0.drop(index = 314, axis = 0)
case_0 = case_0.drop(index = 495, axis = 0)
case_0 = case_0.drop(index = 620, axis = 0)
case_0 = case_0.drop(index = 838, axis = 0)
case_0 = case_0.drop(index = 1187, axis = 0)
case_0 = case_0.drop(index = [31,32], axis = 0)
case_0 = case_0.drop(index = 46, axis = 0)
case_0 = case_0.drop(index = 58, axis = 0)
case_0 = case_0.drop(index = 81, axis = 0)
case_0 = case_0.drop(index = [95,99], axis = 0)
case_0 = case_0.drop(index = [146,1010], axis = 0)
case_0 = case_0.drop(index = 257, axis = 0)
case_0 = case_0.drop(index = 327, axis = 0)
case_0 = case_0.drop(index = 352, axis = 0)
case_0 = case_0.drop(index = 373, axis = 0)
case_0 = case_0.drop(index = 526, axis = 0)
case_0 = case_0.drop(index = 557, axis = 0)
case_0 = case_0.drop(index = 735, axis = 0)
case_0 = case_0.drop(index = [750,973], axis = 0)
case_0 = case_0.drop(index = 863, axis = 0)
case_0 = case_0.drop(index = 910, axis = 0)
case_0 = case_0.drop(index = 924, axis = 0)
case_0 = case_0.drop(index = 950, axis = 0)
case_0 = case_0.drop(index = 1088, axis = 0)
case_0 = case_0.drop(index = 1120, axis = 0)
case_0 = case_0.drop(index = 1199, axis = 0)
case_0 = case_0.drop(index = 60, axis = 0)
case_0 = case_0.drop(index = 329, axis = 0)
case_0 = case_0.drop(index = 912, axis = 0)

In [37]:
case_0 = case_0.drop(index = 1401, axis = 0)

In [38]:
case_0 = case_0.drop(index = 82, axis = 0)

In [39]:
case_0 = case_0.drop(index = 878, axis = 0)

In [40]:
case_0 = case_0.drop(index = 923, axis = 0)

### Feature - 0

In [41]:
case_0_0 = case_0[case_0["feature_num"] == 0]
case_0_0.shape

(90, 11)

In [42]:
pn_Num_list = list(case_0_0.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_0[case_0_0.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_0[case_0_0.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_0.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_0.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(69, 18)

In [43]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_0.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 69/69 [00:00<00:00, 699.35it/s]


In [44]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_0.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 501.67it/s]


In [45]:
!python -m spacy init fill-config ./base_config_0.cfg ./config_0.cfg

[+] Auto-filled config with all values
[+] Saved config
config_0.cfg
You can now add your data and train your pipeline:
python -m spacy train config_0.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:35:48.362147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:35:48.362192: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [46]:
!python -m spacy train config_0.cfg --output ./output_0

[i] Saving to output directory: output_0
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     67.33    0.60    0.32    5.56    0.01
  2     200        499.86   1563.09   85.71   88.24   83.33    0.86
  5     400         47.09     59.64   85.71   88.24   83.33    0.86
  8     600         11.89     11.16   85.71   88.24   83.33    0.86
 11     800         23.35     15.03   77.78   77.78   77.78    0.78
 14    1000          3.09      2.26   85.71   88.24   83.33    0.86
 17    1200          0.01      0.01   77.78   77.78   77.78    0.78
 20    1400          0.00      0.00   77.78   77.78   77.78    0.78
 23    1600        

2022-05-02 13:35:52.998333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:35:52.998384: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:35:55,800] [INFO] Set up nlp object from config
[2022-05-02 13:35:55,807] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:35:55,809] [INFO] Created vocabulary
[2022-05-02 13:35:55,814] [INFO] Finished initializing nlp object
[2022-05-02 13:35:56,244] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 1

In [47]:
case_0_1 = case_0[case_0["feature_num"] == 1]
case_0_1.shape

(90, 11)

In [48]:
pn_Num_list = list(case_0_1.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_1[case_0_1.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_1[case_0_1.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_1.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_1.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(71, 18)

In [49]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_1.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 71/71 [00:00<00:00, 838.15it/s]


In [50]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_1.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 821.04it/s]


In [51]:
!python -m spacy init fill-config ./base_config_1.cfg ./config_1.cfg

[+] Auto-filled config with all values
[+] Saved config
config_1.cfg
You can now add your data and train your pipeline:
python -m spacy train config_1.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:37:49.358525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:37:49.358561: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [52]:
!python -m spacy train config_1.cfg --output ./output_1

[i] Saving to output directory: output_1
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     59.17    0.00    0.00    0.00    0.00
  2     200        257.12   1388.73  100.00  100.00  100.00    1.00
  5     400         17.90     34.02   94.74   90.00  100.00    0.95
  8     600          3.12      5.41   91.89   89.47   94.44    0.92
 11     800          0.00      0.00   91.89   89.47   94.44    0.92
 14    1000          2.33      2.13   94.44   94.44   94.44    0.94
 16    1200          0.00      0.00   94.44   94.44   94.44    0.94
 19    1400          0.01      0.01   91.89   89.47   94.44    0.92
 22    1600        

2022-05-02 13:37:53.486887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:37:53.486929: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:37:56,107] [INFO] Set up nlp object from config
[2022-05-02 13:37:56,114] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:37:56,117] [INFO] Created vocabulary
[2022-05-02 13:37:56,118] [INFO] Finished initializing nlp object
[2022-05-02 13:37:56,509] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 2

In [53]:
case_0_2 = case_0[case_0["feature_num"] == 2]
case_0_2.shape

(70, 11)

In [54]:
pn_Num_list = list(case_0_2.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_2[case_0_2.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_2[case_0_2.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_2.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_2.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(55, 14)

In [55]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_2.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 746.23it/s]


In [56]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_2.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 585.27it/s]


In [57]:
!python -m spacy init fill-config ./base_config_2.cfg ./config_2.cfg

[+] Auto-filled config with all values
[+] Saved config
config_2.cfg
You can now add your data and train your pipeline:
python -m spacy train config_2.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:39:50.710000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:39:50.710042: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [58]:
!python -m spacy train config_2.cfg --output ./output_2

[i] Saving to output directory: output_2
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     50.17    1.48    0.76   33.33    0.01
  3     200        217.60   1161.03   74.07   83.33   66.67    0.74
  7     400         13.67     26.60   75.86   78.57   73.33    0.76
 10     600          0.12      0.15   78.57   84.62   73.33    0.79
 14     800          9.47     11.36   73.33   73.33   73.33    0.73
 18    1000         21.24     17.66   93.33   93.33   93.33    0.93
 21    1200          6.04      4.44   89.66   92.86   86.67    0.90
 25    1400      23482.23    150.14   60.00   60.00   60.00    0.60
 29    1600        

2022-05-02 13:39:55.245537: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:39:55.245574: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:39:58,201] [INFO] Set up nlp object from config
[2022-05-02 13:39:58,208] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:39:58,211] [INFO] Created vocabulary
[2022-05-02 13:39:58,212] [INFO] Finished initializing nlp object
[2022-05-02 13:39:58,573] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 3

In [59]:
case_0_3 = case_0[case_0["feature_num"] == 3]
case_0_3.shape

(118, 11)

In [60]:
pn_Num_list = list(case_0_3.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_3[case_0_3.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_3[case_0_3.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_3.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_3.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(70, 18)

In [61]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_3.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 797.48it/s]


In [62]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_3.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 903.14it/s]


In [63]:
!python -m spacy init fill-config ./base_config_3.cfg ./config_3.cfg

[+] Auto-filled config with all values
[+] Saved config
config_3.cfg
You can now add your data and train your pipeline:
python -m spacy train config_3.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:42:45.082240: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:42:45.082277: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [64]:
!python -m spacy train config_3.cfg --output ./output_3

[i] Saving to output directory: output_3
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     56.83    0.00    0.00    0.00    0.00
  2     200       1446.31   1934.43   15.00   18.75   12.50    0.15
  5     400         38.25    282.24   30.43   31.82   29.17    0.30
  8     600         77.60    184.82   40.00   63.64   29.17    0.40
 11     800         66.52    126.37   53.06   52.00   54.17    0.53
 14    1000         79.47    103.28   37.74   34.48   41.67    0.38
 17    1200         91.51     99.87   45.28   41.38   50.00    0.45
 20    1400         79.47     86.62   45.83   45.83   45.83    0.46
 22    1600        

2022-05-02 13:42:49.167224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:42:49.167266: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:42:52,246] [INFO] Set up nlp object from config
[2022-05-02 13:42:52,253] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:42:52,256] [INFO] Created vocabulary
[2022-05-02 13:42:52,256] [INFO] Finished initializing nlp object
[2022-05-02 13:42:52,640] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 4

In [65]:
case_0_4 = case_0[case_0["feature_num"] == 4]
case_0_4.shape

(93, 11)

In [66]:
pn_Num_list = list(case_0_4.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_4[case_0_4.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_4[case_0_4.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_4.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_4.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(74, 19)

In [67]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_4.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 833.51it/s]


In [68]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_4.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 952.99it/s]


In [69]:
!python -m spacy init fill-config ./base_config_4.cfg ./config_4.cfg

[+] Auto-filled config with all values
[+] Saved config
config_4.cfg
You can now add your data and train your pipeline:
python -m spacy train config_4.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:45:14.842701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:45:14.842742: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [70]:
!python -m spacy train config_4.cfg --output ./output_4

[i] Saving to output directory: output_4
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     49.50    0.00    0.00    0.00    0.00
  2     200         22.37   1141.44   80.00   87.50   73.68    0.80
  5     400        420.37    151.20   80.00   87.50   73.68    0.80
  8     600       1234.01    137.49   86.49   88.89   84.21    0.86
 10     800         36.63     37.56   78.95   78.95   78.95    0.79
 13    1000         19.54     17.86   85.71   93.75   78.95    0.86
 16    1200       1518.82     50.31   74.29   81.25   68.42    0.74
 18    1400         21.90     12.01   80.00   87.50   73.68    0.80
 21    1600       2

2022-05-02 13:45:18.769103: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:45:18.769148: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:45:21,404] [INFO] Set up nlp object from config
[2022-05-02 13:45:21,412] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:45:21,415] [INFO] Created vocabulary
[2022-05-02 13:45:21,416] [INFO] Finished initializing nlp object
[2022-05-02 13:45:21,793] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 5

In [71]:
case_0_5 = case_0[case_0["feature_num"] == 5]
case_0_5.shape

(110, 11)

In [72]:
pn_Num_list = list(case_0_5.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_5[case_0_5.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_5[case_0_5.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_5.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_5.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(86, 22)

In [73]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_5.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 917.77it/s]


In [74]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_5.spacy")

100%|████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 1099.66it/s]


In [75]:
!python -m spacy init fill-config ./base_config_5.cfg ./config_5.cfg

[+] Auto-filled config with all values
[+] Saved config
config_5.cfg
You can now add your data and train your pipeline:
python -m spacy train config_5.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:48:47.941748: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:48:47.941792: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [76]:
!python -m spacy train config_5.cfg --output ./output_5

[i] Saving to output directory: output_5
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     61.67    0.79    0.41   13.64    0.01
  2     200        860.27   1729.01   57.14   60.00   54.55    0.57
  4     400        113.57    176.52   71.11   69.57   72.73    0.71
  6     600        164.39    108.00   59.65   48.57   77.27    0.60
  9     800         40.74     48.10   69.39   62.96   77.27    0.69
 11    1000         47.92     35.34   66.67   65.22   68.18    0.67
 13    1200         56.42     28.26   71.11   69.57   72.73    0.71
 16    1400         19.22      9.37   69.57   66.67   72.73    0.70
 18    1600        

2022-05-02 13:48:51.878345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:48:51.878380: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:48:54,439] [INFO] Set up nlp object from config
[2022-05-02 13:48:54,450] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:48:54,453] [INFO] Created vocabulary
[2022-05-02 13:48:54,454] [INFO] Finished initializing nlp object
[2022-05-02 13:48:54,822] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 6

In [77]:
case_0_6 = case_0[case_0["feature_num"] == 6]
case_0_6.shape

(113, 11)

In [78]:
pn_Num_list = list(case_0_6.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_6[case_0_6.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_6[case_0_6.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_6.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_6.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(72, 19)

In [79]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_6.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 752.52it/s]


In [80]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_6.spacy")

100%|████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 1002.99it/s]


In [81]:
!python -m spacy init fill-config ./base_config_6.cfg ./config_6.cfg

[+] Auto-filled config with all values
[+] Saved config
config_6.cfg
You can now add your data and train your pipeline:
python -m spacy train config_6.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:50:50.955700: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:50:50.955738: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [82]:
!python -m spacy train config_6.cfg --output ./output_6

[i] Saving to output directory: output_6
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     59.83    0.48    0.25    4.76    0.00
  2     200         28.76   1257.48   28.57   35.71   23.81    0.29
  5     400       1194.77    480.44   34.29   42.86   28.57    0.34
  8     600        217.44    150.39   40.00   42.11   38.10    0.40
 11     800         65.57     88.83   43.24   50.00   38.10    0.43
 13    1000        525.55     71.80   40.00   42.11   38.10    0.40
 16    1200         26.65     16.68   22.86   28.57   19.05    0.23
 19    1400         49.20     14.43   41.03   44.44   38.10    0.41
 22    1600        

2022-05-02 13:50:55.085077: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:50:55.085115: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:50:57,777] [INFO] Set up nlp object from config
[2022-05-02 13:50:57,784] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:50:57,787] [INFO] Created vocabulary
[2022-05-02 13:50:57,787] [INFO] Finished initializing nlp object
[2022-05-02 13:50:58,171] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 7

In [83]:
case_0_7 = case_0[case_0["feature_num"] == 7]
case_0_7.shape

(75, 11)

In [84]:
pn_Num_list = list(case_0_7.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_7[case_0_7.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_7[case_0_7.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_7.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_7.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(58, 15)

In [85]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_7.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 58/58 [00:00<00:00, 842.44it/s]


In [86]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_7.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 940.93it/s]


In [87]:
!python -m spacy init fill-config ./base_config_7.cfg ./config_7.cfg

[+] Auto-filled config with all values
[+] Saved config
config_7.cfg
You can now add your data and train your pipeline:
python -m spacy train config_7.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:54:30.812276: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:54:30.812313: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [88]:
!python -m spacy train config_7.cfg --output ./output_7

[i] Saving to output directory: output_7
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     67.50    0.00    0.00    0.00    0.00
  3     200       2816.96   2150.78   93.75   88.24  100.00    0.94
  6     400         14.17     43.02   96.77   93.75  100.00    0.97
 10     600          4.82      5.95   96.77   93.75  100.00    0.97
 13     800          0.00      0.00   96.77   93.75  100.00    0.97
 17    1000          0.00      0.00   96.77   93.75  100.00    0.97
 20    1200          0.00      0.00   96.77   93.75  100.00    0.97
 24    1400          0.00      0.00   96.77   93.75  100.00    0.97
 27    1600        

2022-05-02 13:54:34.715026: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:54:34.715071: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:54:37,274] [INFO] Set up nlp object from config
[2022-05-02 13:54:37,281] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:54:37,284] [INFO] Created vocabulary
[2022-05-02 13:54:37,285] [INFO] Finished initializing nlp object
[2022-05-02 13:54:37,628] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 8

In [89]:
case_0_8 = case_0[case_0["feature_num"] == 8]
case_0_8.shape

(116, 11)

In [90]:
pn_Num_list = list(case_0_8.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_8[case_0_8.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_8[case_0_8.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_8.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_8.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(80, 20)

In [91]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_8.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 882.07it/s]

Skipping entity


In [92]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_8.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 912.15it/s]


In [93]:
!python -m spacy init fill-config ./base_config_8.cfg ./config_8.cfg

[+] Auto-filled config with all values
[+] Saved config
config_8.cfg
You can now add your data and train your pipeline:
python -m spacy train config_8.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:56:34.016023: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:56:34.016062: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [94]:
!python -m spacy train config_8.cfg --output ./output_8

[i] Saving to output directory: output_8
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     50.33    0.00    0.00    0.00    0.00
  2     200         20.23   1358.78   54.55   44.12   71.43    0.55
  5     400         40.73    246.01   56.52   52.00   61.90    0.57
  7     600       2251.95    417.70   44.44   53.33   38.10    0.44
 10     800       2531.32    220.99   50.00   47.83   52.38    0.50
 12    1000        148.46    152.25   43.24   50.00   38.10    0.43
 15    1200        118.01    131.15   41.18   53.85   33.33    0.41
 17    1400        117.48    105.95   50.00   52.63   47.62    0.50
 20    1600        

2022-05-02 13:56:37.908665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:56:37.908708: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:56:40,471] [INFO] Set up nlp object from config
[2022-05-02 13:56:40,478] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:56:40,481] [INFO] Created vocabulary
[2022-05-02 13:56:40,481] [INFO] Finished initializing nlp object
[2022-05-02 13:56:40,872] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 9

In [95]:
case_0_9 = case_0[case_0["feature_num"] == 9]
case_0_9.shape

(160, 11)

In [96]:
pn_Num_list = list(case_0_9.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_9[case_0_9.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_9[case_0_9.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_9.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_9.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(103, 26)

In [97]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_9.spacy")

100%|███████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 854.08it/s]


In [98]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_9.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 788.40it/s]


In [99]:
!python -m spacy init fill-config ./base_config_9.cfg ./config_9.cfg

[+] Auto-filled config with all values
[+] Saved config
config_9.cfg
You can now add your data and train your pipeline:
python -m spacy train config_9.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 13:58:39.740485: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:58:39.740524: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [100]:
!python -m spacy train config_9.cfg --output ./output_9

[i] Saving to output directory: output_9
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     54.17    0.00    0.00    0.00    0.00
  1     200         30.01   1206.73   62.50   54.35   73.53    0.62
  3     400      22040.39    668.00   71.79   63.64   82.35    0.72
  5     600       2026.26    174.60   68.57   66.67   70.59    0.69
  7     800       1315.77    142.60   73.24   70.27   76.47    0.73
  9    1000         73.27     91.95   74.29   72.22   76.47    0.74
 11    1200         76.22     69.59   75.00   80.00   70.59    0.75
 13    1400         79.56     63.21   72.46   71.43   73.53    0.72
 15    1600        

2022-05-02 13:58:43.672796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 13:58:43.672841: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 13:58:46,202] [INFO] Set up nlp object from config
[2022-05-02 13:58:46,209] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 13:58:46,211] [INFO] Created vocabulary
[2022-05-02 13:58:46,212] [INFO] Finished initializing nlp object
[2022-05-02 13:58:46,641] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 10

In [101]:
case_0_10 = case_0[case_0["feature_num"] == 10]
case_0_10.shape

(99, 11)

In [102]:
pn_Num_list = list(case_0_10.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_10[case_0_10.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_10[case_0_10.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_10.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_10.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(73, 19)

In [103]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_10.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 871.24it/s]


In [104]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_10.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 828.79it/s]


In [105]:
!python -m spacy init fill-config ./base_config_10.cfg ./config_10.cfg

[+] Auto-filled config with all values
[+] Saved config
config_10.cfg
You can now add your data and train your pipeline:
python -m spacy train config_10.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 14:03:23.468008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 14:03:23.468050: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [106]:
!python -m spacy train config_10.cfg --output ./output_10

[i] Saving to output directory: output_10

2022-05-02 14:03:27.398933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 14:03:27.398974: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 14:03:29,955] [INFO] Set up nlp object from config
[2022-05-02 14:03:29,962] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 14:03:29,965] [INFO] Created vocabulary
[2022-05-02 14:03:29,965] [INFO] Finished initializing nlp object
[2022-05-02 14:03:30,334] [INFO] Initialized pipeline components: ['tok2vec', 'ner']



[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     60.83    0.00    0.00    0.00    0.00
  2     200       3171.75   2613.93   24.24   30.77   20.00    0.24
  5     400         48.68    251.71   60.87   53.85   70.00    0.61
  8     600        106.84    179.28   55.56   62.50   50.00    0.56
 10     800         89.40    107.85   54.55   50.00   60.00    0.55
 13    1000         71.18     84.02   48.78   47.62   50.00    0.49
 16    1200         76.13     73.56   51.16   47.83   55.00    0.51
 19    1400        532.74     56.43   55.56   62.50   50.00    0.56
 21    1600         60.67     46.53   53.66   52.38   55.00

### Feature 11

In [107]:
case_0_11 = case_0[case_0["feature_num"] == 11]
case_0_11.shape

(94, 11)

In [108]:
pn_Num_list = list(case_0_11.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_11[case_0_11.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_11[case_0_11.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_11.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_11.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(75, 19)

In [109]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_11.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 844.86it/s]


In [110]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_11.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 949.96it/s]


In [111]:
!python -m spacy init fill-config ./base_config_11.cfg ./config_11.cfg

[+] Auto-filled config with all values
[+] Saved config
config_11.cfg
You can now add your data and train your pipeline:
python -m spacy train config_11.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 14:05:26.794909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 14:05:26.794947: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [112]:
!python -m spacy train config_11.cfg --output ./output_11

[i] Saving to output directory: output_11
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     62.00    0.00    0.00    0.00    0.00
  2     200       4985.02   2028.15   89.47   89.47   89.47    0.89
  5     400          8.02     20.88  100.00  100.00  100.00    1.00
  8     600          2.19      3.41  100.00  100.00  100.00    1.00
 10     800          0.00      0.00  100.00  100.00  100.00    1.00
 13    1000          0.00      0.00  100.00  100.00  100.00    1.00
 16    1200          0.00      0.00  100.00  100.00  100.00    1.00
 18    1400          0.30      0.27   92.31   90.00   94.74    0.92
 21    1600       

2022-05-02 14:05:30.676235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 14:05:30.676271: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 14:05:33,258] [INFO] Set up nlp object from config
[2022-05-02 14:05:33,265] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 14:05:33,268] [INFO] Created vocabulary
[2022-05-02 14:05:33,268] [INFO] Finished initializing nlp object
[2022-05-02 14:05:33,638] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


### Feature 12

In [113]:
case_0_12 = case_0[case_0["feature_num"] == 12]
case_0_12.shape

(91, 11)

In [114]:
pn_Num_list = list(case_0_12.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in case_0_12[case_0_12.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = case_0_12[case_0_12.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_12.json", "w") as outfile:
    outfile.write(json_object)
    
# Opening JSON file
f = open("sample_12.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

train, validation = train_test_split(data, test_size = 0.2, random_state = 100)
len(train),len(validation)

(72, 19)

In [115]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train_12.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 650.81it/s]


In [116]:
#Validation data set
camp_validation = create_training(validation)
camp_validation.to_disk("nbme_validation_12.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 952.57it/s]


In [117]:
!python -m spacy init fill-config ./base_config_12.cfg ./config_12.cfg

[+] Auto-filled config with all values
[+] Saved config
config_12.cfg
You can now add your data and train your pipeline:
python -m spacy train config_12.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-02 14:07:30.060842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 14:07:30.060878: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [118]:
!python -m spacy train config_12.cfg --output ./output_12

[i] Saving to output directory: output_12
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     63.33    0.00    0.00    0.00    0.00
  2     200          5.06    958.24  100.00  100.00  100.00    1.00
  5     400          0.00      0.00  100.00  100.00  100.00    1.00
  8     600          0.00      0.00  100.00  100.00  100.00    1.00
 11     800          0.00      0.00  100.00  100.00  100.00    1.00
 13    1000          0.00      0.00  100.00  100.00  100.00    1.00
 16    1200          0.00      0.00  100.00  100.00  100.00    1.00
 19    1400          0.00      0.00  100.00  100.00  100.00    1.00
 22    1600       

2022-05-02 14:07:33.964983: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-02 14:07:33.965019: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-02 14:07:36,537] [INFO] Set up nlp object from config
[2022-05-02 14:07:36,544] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-02 14:07:36,547] [INFO] Created vocabulary
[2022-05-02 14:07:36,548] [INFO] Finished initializing nlp object
[2022-05-02 14:07:36,912] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
